In [1]:
import os
import pandas as pd
import numpy as np
import xarray as xr
from datetime import datetime as dt
from scipy import integrate
import matplotlib.pyplot as plt
year = XXXX  # Change this as needed
month = "YY"  # Change this as needed (ensure two-digit format)
# Load data from NetCDF files
# Construct file paths dynamically
qdata = xr.open_dataset(f"/g/data/rt52/era5/pressure-levels/reanalysis/q/{year}/q_era5_oper_pl_{year}{month}01-{year}{month}28.nc")
q2mdata = xr.open_dataset(f"/g/data/k10/ak6086/q2m/{year}/q2m_{year}{month}.nc")
vdata = xr.open_dataset(f"/g/data/rt52/era5/pressure-levels/reanalysis/v/{year}/v_era5_oper_pl_{year}{month}01-{year}{month}28.nc")
v10mdata = xr.open_dataset(f"/g/data/rt52/era5/single-levels/reanalysis/10v/{year}/10v_era5_oper_sfc_{year}{month}01-{year}{month}28.nc")
spdata = xr.open_dataset(f"/g/data/rt52/era5/single-levels/reanalysis/sp/{year}/sp_era5_oper_sfc_{year}{month}01-{year}{month}28.nc")
time_steps = qdata.time
g=9.81
vIVT_all_timesteps = []
for tt in range(len(time_steps)):
    print(f"Processing time step {tt + 1}/{len(time_steps)}")

    # Extract surface pressure data
    p_s_xr_2d = spdata['sp'][tt, :, :]  # Surface pressure [Pa]
    p_s_2d = p_s_xr_2d.compute().values
    p_s_hPa_2d = p_s_2d / 1e2  # Convert pressure to hPa

    # Extract specific humidity at 2 meters and meridional velocity at 10 meters
    q2m_xr_2d = q2mdata['q2m'][tt, :, :]
    q2m_2d = q2m_xr_2d.compute().values
    v10_xr_2d = v10mdata['v10'][tt, :, :]
    v10_2d = v10_xr_2d.compute().values

    # Extract 3D specific humidity and meridional velocity
    xr_q_3d = qdata['q'][tt, :, :, :]  # Specific humidity [kg/kg]
    q_3d = xr_q_3d.compute().values
    xr_v_3d = vdata['v'][tt, :, :, :]  # Meridional velocity [m/s]
    v_3d = xr_v_3d.compute().values

    # Create copies for high-pressure calculations
    q_hi_3d = np.copy(q_3d)
    q2m_hi_2d = np.copy(q2m_2d)
    v_hi_3d = np.copy(v_3d)
    v10m_hi_2d = np.copy(v10_2d)

    # Set values to zero where surface pressure is less than 1000 hPa
    mask_low_pressure = p_s_hPa_2d < 1000
    q_hi_3d[:, mask_low_pressure] = 0.
    q2m_hi_2d[mask_low_pressure] = 0.
    v_hi_3d[:, mask_low_pressure] = 0.
    v10m_hi_2d[mask_low_pressure] = 0.

    # Reshape 2D fields to 3D for stacking
    q2m_hi_3d = q2m_hi_2d.reshape((1, q2m_hi_2d.shape[0], q2m_hi_2d.shape[1]))
    v10m_hi_3d = v10m_hi_2d.reshape((1, v10m_hi_2d.shape[0], v10m_hi_2d.shape[1]))

    # Stack high-pressure data
    q_hi_3d_stacked = np.append(q_hi_3d, q2m_hi_3d, axis=0)
    v_hi_3d_stacked = np.append(v_hi_3d, v10m_hi_3d, axis=0)

    # Define pressure levels
    p = qdata.level.values  # Pressure levels [hPa]
    p_levels_3d = np.zeros((len(p), q_3d.shape[1], q_3d.shape[2]))
    for i in range(len(p)):
        p_levels_3d[i, :, :] = p[i]

    # Add surface pressure as a new level
    p_s_hPa_3d = p_s_hPa_2d.reshape((1, p_s_hPa_2d.shape[0], p_s_hPa_2d.shape[1]))
    p_hi_3d_stacked = np.append(p_levels_3d, p_s_hPa_3d, axis=0)

    # Slice finite layers for high-pressure calculations
    q_finite_hi = q_hi_3d_stacked[17:, :, :]
    v_finite_hi = v_hi_3d_stacked[17:, :, :]
    p_finite_hi = p_hi_3d_stacked[17:, :, :]

    # Compute mid-layer averages
    q_mid_hi = (q_finite_hi[:-1, :, :] + q_finite_hi[1:, :, :]) / 2
    v_mid_hi = (v_finite_hi[:-1, :, :] + v_finite_hi[1:, :, :]) / 2
    dp_hi = np.abs(p_finite_hi[:-1, :, :] - p_finite_hi[1:, :, :])

    # Calculate vertically integrated vapor transport (high-pressure)
    vIVT_hi_2d = np.nansum(1 * (q_mid_hi * v_mid_hi * dp_hi) / g, axis=0)

    ## For Pressure less than or equal to 1000
    q_lo_3d = np.copy(q_3d)
    q2m_lo_2d = np.copy(q2m_2d)
    v_lo_3d = np.copy(v_3d)
    v10m_lo_2d = np.copy(v10_2d)

    q_lo_3d[:, p_s_hPa_2d > 1000] = 0.
    q2m_lo_2d[p_s_hPa_2d > 1000] = 0.
    v_lo_3d[:, p_s_hPa_2d > 1000] = 0.
    v10m_lo_2d[p_s_hPa_2d > 1000] = 0.

    # Stack the 2-m specific humidity onto the humidity profile array and v10 onto the v
    q2m_lo_3d = q2m_lo_2d.reshape((1, q2m_lo_2d.shape[0], q2m_lo_2d.shape[1]))
    q_lo_3d_stacked = np.append(q_lo_3d, q2m_lo_3d, axis=0)
    v10m_lo_3d = v10m_lo_2d.reshape((1, v10m_lo_2d.shape[0], v10m_lo_2d.shape[1]))
    v_lo_3d_stacked = np.append(v_lo_3d, v10m_lo_3d, axis=0)

    # Stack the surface pressure values onto the pressure levels array
    p_s_hPa_3d = p_s_hPa_2d.reshape((1, p_s_hPa_2d.shape[0], p_s_hPa_2d.shape[1]))
    p_lo_3d_stacked = np.append(p_levels_3d, p_s_hPa_3d, axis=0)
    from numba import jit
    @jit
    def compute_vIVT(q_lo_3d_stacked, v_lo_3d_stacked, p_lo_3d_stacked, p_s_hPa_2d):
        vIVT_lo_2d = np.zeros_like(vIVT_hi_2d)
        for i in range(721):
            for j in range(1440):
                p_new = p_lo_3d_stacked[17:, i, j][p_lo_3d_stacked[17:, i, j] <= p_s_hPa_2d[i, j]]
                q_new = q_lo_3d_stacked[17:, i, j][p_lo_3d_stacked[17:, i, j] <= p_s_hPa_2d[i, j]]
                v_new = v_lo_3d_stacked[17:, i, j][p_lo_3d_stacked[17:, i, j] <= p_s_hPa_2d[i, j]]

                # Trapezoidal integration
                q_mid_lo = (q_new[:-1] + q_new[1:]) / 2.
                v_mid_lo = (v_new[:-1] + v_new[1:]) / 2.
                dp_lo = np.abs(p_new[:-1] - p_new[1:])
                vIVT_lo_2d[i, j] = np.sum(1 * (q_mid_lo * v_mid_lo * dp_lo) / g, axis=0)

        return vIVT_lo_2d

    vIVT_lo_2d = compute_vIVT(q_lo_3d_stacked, v_lo_3d_stacked, p_lo_3d_stacked, p_s_hPa_2d)
    vIVT = (vIVT_hi_2d + vIVT_lo_2d) * 100
    vIVT_all_timesteps.append(vIVT)

vIVT_all_timesteps = np.array(vIVT_all_timesteps)
print("Processing complete. vIVT computed for all time steps.")
# Create a DataFrame for coordinates (latitude, longitude) and time
latitude = qdata.latitude.values  # Assuming latitude values are available in the qdata
longitude = qdata.longitude.values  # Assuming longitude values are available in the qdata
time = qdata.time.values  # Time steps from the dataset

# Create the dataset for vIVT
vivt_dataset = xr.Dataset(
    {
        'latitude': (['latitude'], latitude),
        'longitude': (['longitude'], longitude),
        'time': (['time'], time),
        'vivt': (['time', 'latitude', 'longitude'], vIVT_all_timesteps),
    }
)

# Specify the path to save the NetCDF file
vivt_nc_file = f"/g/data/k10/ak6086/vIVTsur/{year}/vivt{year}{month}.nc"

# Save to NetCDF
vivt_dataset.to_netcdf(vivt_nc_file, format='NETCDF4')

print("vIVT data saved successfully to NetCDF file.")

/home/565/ak6086/micromamba/envs/PhDAnalysis/lib/python3.12/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


Processing time step 1/672
Processing time step 2/672
Processing time step 3/672
Processing time step 4/672
Processing time step 5/672
Processing time step 6/672
Processing time step 7/672
Processing time step 8/672
Processing time step 9/672
Processing time step 10/672
Processing time step 11/672
Processing time step 12/672
Processing time step 13/672
Processing time step 14/672
Processing time step 15/672
Processing time step 16/672
Processing time step 17/672
Processing time step 18/672
Processing time step 19/672
Processing time step 20/672
Processing time step 21/672
Processing time step 22/672
Processing time step 23/672
Processing time step 24/672
Processing time step 25/672
Processing time step 26/672
Processing time step 27/672
Processing time step 28/672
Processing time step 29/672
Processing time step 30/672
Processing time step 31/672
Processing time step 32/672
Processing time step 33/672
Processing time step 34/672
Processing time step 35/672
Processing time step 36/672
P